In [2]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit
from plotnine import *
import sklearn.ensemble
import sklearn.model_selection
import sklearn.metrics
import sklearn.linear_model
import matplotlib.pylab as plt
from clean_data import *

In [3]:
df = pd.read_csv("data/loan_level_500k.csv", low_memory=False, index_col=19)
df.head()

,CREDIT_SCORE,FIRST_PAYMENT_DATE,FIRST_TIME_HOMEBUYER_FLAG,MATURITY_DATE,METROPOLITAN_STATISTICAL_AREA,MORTGAGE_INSURANCE_PERCENTAGE,NUMBER_OF_UNITS,OCCUPANCY_STATUS,ORIGINAL_COMBINED_LOAN_TO_VALUE,ORIGINAL_DEBT_TO_INCOME_RATIO,...,PROPERTY_STATE,PROPERTY_TYPE,POSTAL_CODE,LOAN_PURPOSE,ORIGINAL_LOAN_TERM,NUMBER_OF_BORROWERS,SELLER_NAME,SERVICER_NAME,PREPAID,DELINQUENT
LOAN_SEQUENCE_NUMBER,,,,,,,,,,,,,,,,,,,,,
F199Q1000004,669.0,200206,N,202901,NaN,0.0,1.0,O,80.0,33.0,...,WV,SF,26100.0,P,320,2.0,Other sellers,Other servicers,True,False
F199Q1000005,732.0,199904,N,202903,17140.0,0.0,1.0,O,25.0,10.0,...,OH,SF,45200.0,N,360,1.0,Other sellers,Other servicers,True,False
F199Q1000007,679.0,200208,N,202902,15940.0,30.0,1.0,O,91.0,48.0,...,OH,SF,44700.0,P,319,1.0,Other sellers,Other servicers,True,False
F199Q1000013,721.0,200209,N,202902,38060.0,0.0,1.0,O,39.0,13.0,...,AZ,SF,85200.0,N,318,2.0,Other sellers,Other servicers,True,False
F199Q1000015,618.0,200210,N,202902,10420.0,25.0,1.0,O,85.0,24.0,...,OH,SF,44200.0,N,317,2.0,Other sellers,Other servicers,True,False


In [4]:
df = tidy_data(df)
df.head()

,CREDIT_SCORE,FIRST_TIME_HOMEBUYER_FLAG,METROPOLITAN_STATISTICAL_AREA,MORTGAGE_INSURANCE_PERCENTAGE,NUMBER_OF_UNITS,OCCUPANCY_STATUS,ORIGINAL_COMBINED_LOAN_TO_VALUE,ORIGINAL_DEBT_TO_INCOME_RATIO,ORIGINAL_UPB,ORIGINAL_LOAN_TO_VALUE,...,ORIGINAL_LOAN_TERM,NUMBER_OF_BORROWERS,SELLER_NAME,SERVICER_NAME,PREPAID,DELINQUENT,MATURITY_YEAR,MATURITY_MON,FIRST_PAYMENT_YEAR,FIRST_PAYMENT_MON
LOAN_SEQUENCE_NUMBER,,,,,,,,,,,,,,,,,,,,,
F199Q1000004,669.0,N,NaN,0.0,1.0,O,80.0,33.0,162000,80.0,...,320,2.0,Other sellers,Other servicers,True,False,2029,1,2002,6
F199Q1000005,732.0,N,17140.0,0.0,1.0,O,25.0,10.0,53000,25.0,...,360,1.0,Other sellers,Other servicers,True,False,2029,3,1999,4
F199Q1000007,679.0,N,15940.0,30.0,1.0,O,91.0,48.0,133000,91.0,...,319,1.0,Other sellers,Other servicers,True,False,2029,2,2002,8
F199Q1000013,721.0,N,38060.0,0.0,1.0,O,39.0,13.0,174000,39.0,...,318,2.0,Other sellers,Other servicers,True,False,2029,2,2002,9
F199Q1000015,618.0,N,10420.0,25.0,1.0,O,85.0,24.0,122000,85.0,...,317,2.0,Other sellers,Other servicers,True,False,2029,2,2002,10


In [5]:
df['DELINQUENT_ENCODING'] = [1 if i else 0 for i in df['DELINQUENT']]
df.head()

,CREDIT_SCORE,FIRST_TIME_HOMEBUYER_FLAG,METROPOLITAN_STATISTICAL_AREA,MORTGAGE_INSURANCE_PERCENTAGE,NUMBER_OF_UNITS,OCCUPANCY_STATUS,ORIGINAL_COMBINED_LOAN_TO_VALUE,ORIGINAL_DEBT_TO_INCOME_RATIO,ORIGINAL_UPB,ORIGINAL_LOAN_TO_VALUE,...,NUMBER_OF_BORROWERS,SELLER_NAME,SERVICER_NAME,PREPAID,DELINQUENT,MATURITY_YEAR,MATURITY_MON,FIRST_PAYMENT_YEAR,FIRST_PAYMENT_MON,DELINQUENT_ENCODING
LOAN_SEQUENCE_NUMBER,,,,,,,,,,,,,,,,,,,,,
F199Q1000004,669.0,N,NaN,0.0,1.0,O,80.0,33.0,162000,80.0,...,2.0,Other sellers,Other servicers,True,False,2029,1,2002,6,0
F199Q1000005,732.0,N,17140.0,0.0,1.0,O,25.0,10.0,53000,25.0,...,1.0,Other sellers,Other servicers,True,False,2029,3,1999,4,0
F199Q1000007,679.0,N,15940.0,30.0,1.0,O,91.0,48.0,133000,91.0,...,1.0,Other sellers,Other servicers,True,False,2029,2,2002,8,0
F199Q1000013,721.0,N,38060.0,0.0,1.0,O,39.0,13.0,174000,39.0,...,2.0,Other sellers,Other servicers,True,False,2029,2,2002,9,0
F199Q1000015,618.0,N,10420.0,25.0,1.0,O,85.0,24.0,122000,85.0,...,2.0,Other sellers,Other servicers,True,False,2029,2,2002,10,0


In [6]:
#Function from Project 4
def get_roc_data(model, cv_obj):
    curve_df = None
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    
    for i, (train, test) in enumerate(cv_obj.split(X, y)):
        model.fit(X[train], y[train])
        scores = model.predict_proba(X[test])[:,1]
        fpr, tpr, _ = sklearn.metrics.roc_curve(y[test],scores)
        
        interp_tpr = np.interp(mean_fpr, fpr, tpr)
        interp_tpr[0] = 0.0
        tmp = pd.DataFrame({'fold':i, 'fpr': mean_fpr, 'tpr': interp_tpr})
        curve_df = tmp if curve_df is None else pd.concat([curve_df, tmp])
        
        aucs.append(sklearn.metrics.auc(fpr, tpr))
        
    curve_df = curve_df.groupby('fpr').agg({'tpr': 'mean'}).reset_index()
    curve_df.iloc[-1,1] = 1.0
    
    auc_df = pd.DataFrame({'fold': np.arange(len(aucs)), 'auc': aucs})
    return curve_df, auc_df

In [7]:
#Logistic Regression Prep
cv_obj = sklearn.model_selection.StratifiedKFold(n_splits=10)

lr = sklearn.linear_model.LogisticRegression()
parameters = {}

lr_cv = sklearn.model_selection.GridSearchCV(lr, parameters, cv=10)

In [8]:
#Using Function from clean_data script to split into testing and training data
X, y, _ = get_train_test_split_for_ml(df, 2000, )

In [9]:
#Using Function From Project 4
lr_curve_df, lr_auc_df = get_roc_data(lr_cv, cv_obj)

lr_curve_df['model'] = 'logistic regression'
lr_auc_df['model'] = 'logistic regression'

ValueError: Found input variables with inconsistent numbers of samples: [260725, 239412]